## EMotion Detection from Text Data

We have taken the dataset from the Kaggle https://www.kaggle.com/c/sa-emotions

### Load the Libraries

In [8]:
import re
import pandas as pd
import numpy as np
import sklearn
import gensim
import nltk
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,LSTM, Dense, SpatialDropout1D
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from gensim.models import KeyedVectors

In [ ]:
#en_model = KeyedVectors.load_word2vec_format('model/wiki.en/wiki.en.vec')

### Read the dataset

In [22]:
import unicodedata

In [26]:
a='á'
a.encode('ascii','ignore')

b''

In [75]:
data_raw = pd.read_csv('data/isear.csv', error_bad_lines=False,sep="|")
data =pd.DataFrame({'content':data_raw['SIT'],'sentiment':data_raw['Field2']})

In [30]:
#data.groupby('sentiment').count()
#emotions_class=['surprise','hate']#, 'worry','sadness', 'neutral','happiness']
#data = data[data['sentiment'].isin(emotions_class)]

## Text Cleaning

In [76]:
%%time
word_counts = {}
def clean_text(text):
    text= re.sub('[^a-zA-z0-9\s]','',text)
    stop_words_list = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    #extra replace fro removing \\ characters
    words = [re.sub("\\\\",'',word.lower()) for word in words if word.lower() not in stop_words_list and word.isalpha()]   #filter(lambda word: word not in stop_words_list, text.split())
    words = [lemmatizer.lemmatize(word) for word in words]
    for word in words:
        if word not in word_counts:
            word_counts[word] = 1
        else:
            word_counts[word] += 1 
    #word_counts = word_counts_sorted
    #text=" ".join(words)
    return text

def create_word_index(word_count_sorted):
    word_counts = {}
    word_to_index = {}
    index_to_word = {}
    for wc in word_count_sorted:
        word = wc[0]
        word_counts[word] = wc[1] 
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)
            index_to_word[len(word_to_index)] = word
    return word_counts, word_to_index, index_to_word

def feature_mapping(texts, word_to_index, word_counts):
    feature_vectors=np.zeros((len(texts),len(word_to_index)))
    i=0
    for text in texts.values:
        words = nltk.word_tokenize(text)
        for word in words:
            if word in word_to_index:
                feature_vectors[i,word_to_index[word]]=word_counts[word]
        i+=1
    return feature_vectors 

def label_encoder(labels):
    """
    This method fits a label encoder on the string labels.
    """
    label_encoder = LabelEncoder()
    label_encoder.fit(labels)
    return label_encoder.transform(labels)

Wall time: 0 ns


In [77]:
%%time
data['content']=data['content'].apply(clean_text)
word_counts_sorted = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

Wall time: 3.86 s


In [91]:
word_counts_sorted = word_counts_sorted[0:3500]
word_counts, word_to_index, index_to_word = create_word_index(word_counts_sorted)

In [92]:
%%time
feature_vectors = feature_mapping(data['content'],word_to_index,word_counts)

Wall time: 1.11 s


In [93]:
feature_vectors.shape

(7666, 3500)

In [94]:
labels = label_encoder(data['sentiment'])
X_train, X_valid, Y_train, Y_valid = train_test_split(feature_vectors, labels, test_size = 0.2, random_state = 36)
#clf = MultinomialNB()

In [95]:
clf = MultinomialNB()
clf.fit(X_train, Y_train)
pred = clf.predict(X_valid) 
clf.score(X_valid, Y_valid)

0.30182529335071706

In [99]:
%%time
kernel = 'rbf'
clf = svm.SVC(kernel=kernel, gamma=1.5)
clf.fit(X_train[0:4500], Y_train[0:4500])
y_pred_train = clf.predict(X_train)
print(classification_report(y_pred_train, Y_train))

              precision    recall  f1-score   support

           0       0.91      0.79      0.85       164
           1       0.70      1.00      0.82       301
           2       0.73      0.99      0.84      1181
           3       0.99      0.66      0.79      3132
           4       0.72      0.99      0.83      1354

   micro avg       0.82      0.82      0.82      6132
   macro avg       0.81      0.89      0.83      6132
weighted avg       0.86      0.82      0.81      6132

Wall time: 4min 37s


In [77]:
%%time
max_num_words=5000
feature_vector = map_to_features(max_num_words, data['content'])

Wall time: 1.22 s


In [78]:
label_encoder = fit_label_indexer(data['sentiment'])
labels = label_indexing(label_encoder, data['sentiment'])
one_hot_encoder = fit_one_hot_encoder(labels)
labels = one_hot_encode(one_hot_encoder, labels)

C:\Anaconda3\envs\NLP\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [79]:
X_train, X_valid, Y_train, Y_valid = train_test_split(feature_vector,labels, test_size = 0.2, random_state = 36)

In [81]:
embed_dim = 128
lstm_size = 200
feature_vector_size = feature_vector.shape[1]
batch_size=32

model = Sequential()
model.add(Embedding(max_num_words, embed_dim, input_length=feature_vector_size))
model.add(SpatialDropout1D(0.7))
model.add(LSTM(lstm_size, dropout = 0.6, recurrent_dropout = 0.6))
model.add(Dense(13, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 128)          640000    
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 500, 128)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 200)               263200    
_________________________________________________________________
dense_5 (Dense)              (None, 13)                2613      
Total params: 905,813
Trainable params: 905,813
Non-trainable params: 0
_________________________________________________________________
None


In [75]:
model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), batch_size =512, epochs = 100,  verbose = 1)

Train on 24000 samples, validate on 6000 samples
Epoch 1/100
24000/24000 [==============================] - 37s 2ms/step - loss: 2.2034 - acc: 0.2391 - val_loss: 2.0902 - val_acc: 0.2622
Epoch 2/100
24000/24000 [==============================] - 38s 2ms/step - loss: 2.0992 - acc: 0.2703 - val_loss: 2.0761 - val_acc: 0.2777
Epoch 3/100
24000/24000 [==============================] - 37s 2ms/step - loss: 2.0792 - acc: 0.2811 - val_loss: 2.0599 - val_acc: 0.2858
Epoch 4/100
24000/24000 [==============================] - 37s 2ms/step - loss: 2.0571 - acc: 0.2938 - val_loss: 2.0273 - val_acc: 0.2902
Epoch 5/100
24000/24000 [==============================] - 36s 2ms/step - loss: 2.0264 - acc: 0.3020 - val_loss: 1.9901 - val_acc: 0.3028
Epoch 6/100
24000/24000 [==============================] - 40s 2ms/step - loss: 1.9946 - acc: 0.3104 - val_loss: 1.9697 - val_acc: 0.3087
Epoch 7/100
24000/24000 [==============================] - 41s 2ms/step - loss: 1.9745 - acc: 0.3192 - val_loss: 1.9573 - v

KeyboardInterrupt: 